In [1]:
import zipfile
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/cs532/cnn-pruning-status_200-main.zip")
zip_ref.extractall("content/tmp")
zip_ref.close()

In [3]:
import torch
import copy
import torch.nn.utils.prune as prune
import math
from torchvision import datasets,transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

In [4]:
%cd content/tmp/cnn-pruning-status_200-main

/content/content/tmp/cnn-pruning-status_200-main


In [5]:
# Load the pretrained model
from cifar10_models.resnet import resnet18
my_model = resnet18()

# Pretrained model
my_model = resnet18(pretrained=True)
my_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
#calculate sparsity ratio
def calculate_sparsity(model):
  total_count = 0
  zero_count = 0
  
  for buffer_name, buffer in model.named_buffers():
    zero_count += torch.sum(buffer == 0).item()
    if zero_count>0:
      total_count += buffer.nelement()

  print("Total params: ", total_count)
  print("Zero params: ", zero_count)
  return (math.ceil(zero_count*100/total_count))

In [7]:
print("Preparing_data ======>")
transform_train = transforms.Compose([
                                    transforms.RandomCrop(32,padding=4),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))
])
                                
trainset = torchvision.datasets.CIFAR10(
    root = './data',train=True,download=True,transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=128,shuffle=True,num_workers=2)

Preparing_data ======>


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [8]:
loss_history = []
correct_history = []
val_loss_history = []
criterion = nn.CrossEntropyLoss()

In [9]:
def get_optimizer(model):
  optimizer = optim.SGD(model.parameters(), lr=0.1,momentum=0,weight_decay=5e-4)
  return optimizer

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
def train(epoch,model):
  model.train()
  train_acc = 0
  train_loss = 0
  correct = 0
  total = 0

  for batch_idx, (inputs, targets) in enumerate(trainloader):
    inputs,targets = inputs.to(device),targets.to(device)
    outputs = model(inputs)
    loss = criterion(outputs,targets)
    optimizer = get_optimizer(model)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _,predicted = outputs.max(1)
    total +=targets.size(0)
    correct += predicted.eq(targets).sum().item()
    train_acc = 100 *correct/total
  if epoch % 5 == 1:
    print("Training")
    print('[%d/%d] Loss: %.3f | Acc: %.3f (%d/%d)' %(batch_idx,len(trainloader),train_loss/(batch_idx+1),100.*correct/total,correct,total))
    print('Current learning rate is: {}'.format(optimizer.param_groups[0]['lr']))
  loss_history.append(train_loss)
  correct_history.append(train_acc)

  return min(loss_history),max(correct_history)

In [21]:
epochs = 11
# Build and save the iterative pruning model with 50 % sparsity ratio
model1 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model1.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))
for i in range(5):
  print(f"Iteration *********************************************** {i+1} **************************************************" )
  prune.global_unstructured(
  parameters_to_prune,
  pruning_method=prune.L1Unstructured,
  amount=0.128,
  )
  model1= model1.to(device)
  for e in range(epochs):
    if(e+1)%5==1:
      print(f"Epoch ======> {e+1}")
    train(e+1,model1)
    torch.save(model1, "cifar10_models/state_dicts/iterative_50.pt")
print("Sparsity of model1", calculate_sparsity(model1))

Layers to prune 21
Iteration *********************************************** 1 **************************************************
Epoch ======> 1
Training
[390/391] Loss: 1.577 | Acc: 42.432 (21216/50000)
Current learning rate is: 0.1
Epoch ======> 6
Training
[390/391] Loss: 0.582 | Acc: 80.070 (40035/50000)
Current learning rate is: 0.1
Epoch ======> 11
Training
[390/391] Loss: 0.418 | Acc: 85.514 (42757/50000)
Current learning rate is: 0.1
Iteration *********************************************** 2 **************************************************
Epoch ======> 1
Training
[390/391] Loss: 0.394 | Acc: 86.538 (43269/50000)
Current learning rate is: 0.1
Epoch ======> 6
Training
[390/391] Loss: 0.315 | Acc: 89.184 (44592/50000)
Current learning rate is: 0.1
Epoch ======> 11
Training
[390/391] Loss: 0.271 | Acc: 90.556 (45278/50000)
Current learning rate is: 0.1
Iteration *********************************************** 3 **************************************************
Epoch ======> 1


In [22]:
Train and save the final model
epochs = 51
model1= model1.to(device)
  for e in range(epochs):
    if(e+1)%5==1:
      print(f"Epoch ======> {e+1}")
    train(e+1,model1)
    torch.save(model1, "cifar10_models/state_dicts/iterative_50.pt")
torch.save(model1, "cifar10_models/state_dicts/iterative_50.pt")

In [24]:
# Build and save the iterative pruning model with 75 % sparsity ratio
model2 = copy.deepcopy(my_model)

parameters_to_prune =[]
for module_name, module in model2.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))
for i in range(5):
  print(f"Iteration *********************************************** {i+1} **************************************************" )
  prune.global_unstructured(
  parameters_to_prune,
  pruning_method=prune.L1Unstructured,
  amount=0.24,
  )
  model2= model2.to(device)
  for e in range(epochs):
    if(e+1)%5==1:
      print(f"Epoch ======> {e+1}")
    train(e+1,model2)
    torch.save(model2, "cifar10_models/state_dicts/iterative_75.pt")
print("Sparsity of model2", calculate_sparsity(model2))

Layers to prune 21
Iteration *********************************************** 1 **************************************************
Epoch ======> 1
Training
[390/391] Loss: 1.615 | Acc: 40.758 (20379/50000)
Current learning rate is: 0.1
Epoch ======> 6
Training
[390/391] Loss: 0.592 | Acc: 79.580 (39790/50000)
Current learning rate is: 0.1
Epoch ======> 11
Training
[390/391] Loss: 0.425 | Acc: 85.442 (42721/50000)
Current learning rate is: 0.1
Iteration *********************************************** 2 **************************************************
Epoch ======> 1
Training
[390/391] Loss: 0.393 | Acc: 86.552 (43276/50000)
Current learning rate is: 0.1
Epoch ======> 6
Training
[390/391] Loss: 0.326 | Acc: 88.726 (44363/50000)
Current learning rate is: 0.1
Epoch ======> 11
Training
[390/391] Loss: 0.269 | Acc: 90.696 (45348/50000)
Current learning rate is: 0.1
Iteration *********************************************** 3 **************************************************
Epoch ======> 1


In [25]:
# Train and save the final model
epochs = 51
model2= model2.to(device)
  for e in range(epochs):
    if(e+1)%5==1:
      print(f"Epoch ======> {e+1}")
    train(e+1,model2)
    torch.save(model2, "cifar10_models/state_dicts/iterative_75.pt")
torch.save(model2, "cifar10_models/state_dicts/iterative_75.pt")

In [26]:
# Build and save the iterative pruning model with 90 % sparsity ratio
model3 = copy.deepcopy(my_model)
parameters_to_prune =[]
for module_name, module in model3.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, "weight"))
print("Layers to prune {}".format((len(parameters_to_prune))))
for i in range(5):
  print(f"Iteration *********************************************** {i+1} **************************************************" )
  prune.global_unstructured(
  parameters_to_prune,
  pruning_method=prune.L1Unstructured,
  amount=0.36,
  )
  model3= model3.to(device)
  for e in range(epochs):
    if(e+1)%5==1:
      print(f"Epoch ======> {e+1}")
    train(e+1,model3)
    torch.save(model3, "cifar10_models/state_dicts/iterative_90.pt")
print("Sparsity of model3", calculate_sparsity(model3))


Layers to prune 21
Iteration *********************************************** 1 **************************************************
Epoch ======> 1
Training
[390/391] Loss: 1.677 | Acc: 38.346 (19173/50000)
Current learning rate is: 0.1
Epoch ======> 6
Training
[390/391] Loss: 0.618 | Acc: 78.816 (39408/50000)
Current learning rate is: 0.1
Epoch ======> 11
Training
[390/391] Loss: 0.450 | Acc: 84.438 (42219/50000)
Current learning rate is: 0.1
Iteration *********************************************** 2 **************************************************
Epoch ======> 1
Training
[390/391] Loss: 0.423 | Acc: 85.380 (42690/50000)
Current learning rate is: 0.1
Epoch ======> 6
Training
[390/391] Loss: 0.350 | Acc: 87.926 (43963/50000)
Current learning rate is: 0.1
Epoch ======> 11
Training
[390/391] Loss: 0.296 | Acc: 89.600 (44800/50000)
Current learning rate is: 0.1
Iteration *********************************************** 3 **************************************************
Epoch ======> 1


In [27]:
#save the model
epochs = 51
model3= model3.to(device)
  for e in range(epochs):
    if(e+1)%5==1:
      print(f"Epoch ======> {e+1}")
    train(e+1,model3)
    torch.save(model3, "cifar10_models/state_dicts/iterative_90.pt")
torch.save(model3, "cifar10_models/state_dicts/iterative_90.pt")
